# Demo of Classification

This notebook demonstrates the code necessary to train different classifiers and use these to predict new MSMS spectra of either being of "interest" or "other". 

## Setup

In [1]:
from AnnoMe.Classification import (
    generate_ms2deepscore_embeddings,
    add_mzmine_metainfos,
    add_sirius_fingerprints,
    add_sirius_canopus,
    add_sirius_predictions,
    add_mzmine_quant,
    remove_invalid_CEs,
    show_dataset_overview,
    generate_embedding_plots,
    train_and_classify,
    generate_prediction_overview,
    generate_ml_metrics_overview,
    set_random_seeds,
)

import pandas as pd

from collections import OrderedDict
import os

set_random_seeds(42)

## Parameters

In [2]:
# fmt: off
# parameters

# Main folder for input and output files
base_folder = "../../../"

# Path to the MS2DeepScore model file
model_file_name = f"{base_folder}/models/ms2deepscore_model.pt"

# Output directory
output_dir = f"{base_folder}/output_PrenylatedCompounds_BOKUDBs/"

# Main datasets to process for classification
datasets = OrderedDict(
    [
        ## MS/MS of reference prenylated flavones
        #(
        #    "prenylated_flavones_CIDpos_relevant",
        #    {
        #        "name": "prenylated flavonoids",
        #        "type": "train - relevant",
        #        "file": f"{base_folder}/../results/CID_pos__sirius.mgf",
        #        "fragmentation_method": "fragmentation_method",
        #        "colour": "#D41F11",
        #        "canopus_file": f"{base_folder}/../results/CID_pos__sirius/canopus_formula_summary.tsv",
        #    },
        #),
        #(
        #    "prenylated_flavones_CIDneg_relevant",
        #    {
        #        "name": "prenylated flavonoids",
        #        "type": "train - relevant",
        #        "file": f"{base_folder}/../results/CID_neg__sirius.mgf",
        #        "fragmentation_method": "fragmentation_method",
        #        "colour": "#F37A00",
        #        "canopus_file": f"{base_folder}/../results/CID_neg__sirius/canopus_formula_summary.tsv",
        #    },
        #),
        (
            "prenylated_flavones_HCDpos_relevant",
            {
                "name": "prenylated flavonoids",
                "type": "train - relevant",
                "file": f"{base_folder}/../results/HCD_pos__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#10ADC2",
                "canopus_file": f"{base_folder}/../results/HCD_pos__sirius/canopus_formula_summary.tsv",
            },
        ),
        (
            "prenylated_flavones_HCDneg_relevant",
            {
                "name": "prenylated flavonoids",
                "type": "train - relevant",
                "file": f"{base_folder}/../results/HCD_neg__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#017192",
                "canopus_file": f"{base_folder}/../results/HCD_neg__sirius/canopus_formula_summary.tsv",
            },
        ),

        ## MS/MS of wheat samples
        (
            "Wheat_HCDpos_other",
            {
                "name": "wheat metabolites",
                "type": "train - other",
                "file": f"{base_folder}/../results/Wheat_pos__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#F37A00",
            },
        ),
        (
            "Wheat_HCDneg_other",
            {
                "name": "wheat metabolites",
                "type": "train - other",
                "file": f"{base_folder}/../results/Wheat_neg__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#F37A00",
            },
        ),

        ## MS/MS of BOKU MassBank
        (
            "MBBOKU_relevant",
            {
                "name": "MassBank BOKU relevant",
                "type": "train - relevant",
                "file": f"{base_folder}/data/derived/BOKU_iBAM_MB___StructureOfInterest__MatchingSmiles.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#FF0000",
                "fingerprintFile": "::SIRIUS",
            },
        ),
        (
            "MBBOKU_other",
            {
                "name": "MassBank BOKU other",
                "type": "train - other",
                "file": f"{base_folder}/data/derived/BOKU_iBAM_MB___StructureOfInterest__NonMatchingSmiles.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#8B8989",
                "fingerprintFile": "::SIRIUS",
            },
        ),

        ## MS/MS for inference
        #(
        #    "Samp_PaulowinaTomentosa_neg_inference",
        #    {
        #        "name": "Samp_PaulowinaTomentosa_neg",
        #        "type": "inference",
        #        "file": f"{base_folder}/../results/Samp1_neg__sirius.mgf",
        #        "fragmentation_method": "fragmentation_method",
        #        "colour": "#80BF02",
        #        "fingerprintFile": f"{base_folder}/../results/Samp1_neg__sirius_fingerprints.json",
        #        "canopus_file": f"{base_folder}/../results/Samp1_neg__sirius/canopus_formula_summary.tsv",
        #        "quant_file": f"{base_folder}/../results/Samp1_neg__full_feature_table.csv",
        #        "sirius_file": f"{base_folder}/../results/Samp1_neg__sirius/structure_identifications_top-15.tsv",
        #        "mzmine_meta_table": f"{base_folder}/../results/Samp1_neg__full_feature_table.csv",
        #    },
        #),
        (
            "PT22CH_pos_inference",
            {
                "name": "PT22CH_pos",
                "type": "inference",
                "file": f"{base_folder}/../results/PT22CH_pos__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#80BF02",
            },
        ),
        (
            "PT22CH_neg_inference",
            {
                "name": "PT22CH_neg",
                "type": "inference",
                "file": f"{base_folder}/../results/PT22CH_neg__sirius.mgf",
                "fragmentation_method": "fragmentation_method",
                "colour": "#80BF02",
            },
        ),
    ]
)

# meta-data to add to the output from the MS/MS spectra
data_to_add = OrderedDict(
    [
        ("name", ["feature_id", "name", "title", "compound_name"]),
        ("formula", ["formula"]),
        ("smiles", ["smiles"]),
        ("adduct", ["adduct", "precursor_type"]),
        ("ionMode", ["ionmode"]),
        ("RTINSECONDS", ["rtinseconds", "retention_time"]),
        ("precursor_mz", ["pepmass", "precursor_mz"]),
        ("fragmentation_method", ["fragmentation_method", "fragmentation_mode"]),
        ("CE", ["collision_energy"]),
    ]
)

training_subsets = {
    ## "all"           : lambda x: True,
    ## "CE30"          : lambda x: x["CE"] == "30.0",
    ## "CE50"          : lambda x: x["CE"] == "50.0",
    ## "CE70"          : lambda x: x["CE"] == "70.0",
    ## "pos"           : lambda x: x["ionMode"] == "positive",
    ## "neg"           : lambda x: x["ionMode"] == "negative",
    ## "cid"           : lambda x: x["fragmentation_method"] == "cid",
    ## "hcd"           : lambda x: x["fragmentation_method"] == "hcd",
    ## "pos_CE30"      : lambda x: (x["ionMode"] == "positive") & (x["CE"] == "30.0"),
    ## "pos_CE50"      : lambda x: (x["ionMode"] == "positive") & (x["CE"] == "50.0"),
    ## "pos_CE70"      : lambda x: (x["ionMode"] == "positive") & (x["CE"] == "70.0"),
    ## "neg_CE30"      : lambda x: (x["ionMode"] == "negative") & (x["CE"] == "30.0"),
    ## "neg_CE50"      : lambda x: (x["ionMode"] == "negative") & (x["CE"] == "50.0"),
    ## "neg_CE70"      : lambda x: (x["ionMode"] == "negative") & (x["CE"] == "70.0"),
    ## "cid_CE30"      : lambda x: (x["fragmentation_method"] == "cid") & (x["CE"] == "30.0"),
    ## "cid_CE50"      : lambda x: (x["fragmentation_method"] == "cid") & (x["CE"] == "50.0"),
    ## "cid_CE70"      : lambda x: (x["fragmentation_method"] == "cid") & (x["CE"] == "70.0"),
    ## "hcd_CE30"      : lambda x: (x["fragmentation_method"] == "hcd") & (x["CE"] == "30.0"),
    ## "hcd_CE50"      : lambda x: (x["fragmentation_method"] == "hcd") & (x["CE"] == "50.0"),
    ## "hcd_CE70"      : lambda x: (x["fragmentation_method"] == "hcd") & (x["CE"] == "70.0"),
    ## "cid_pos"       : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "positive"),
    ## "cid_neg"       : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "negative"),
    ## "hcd_pos"       : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "positive"),
    ## "hcd_neg"       : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative"),
    ## "cid_pos_CE30"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "positive") & (x["CE"] == "30.0"),
    ## "cid_pos_CE50"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "positive") & (x["CE"] == "50.0"),
    ## "cid_pos_CE70"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "positive") & (x["CE"] == "70.0"),
    ## "cid_neg_CE30"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "negative") & (x["CE"] == "30.0"),
    ## "cid_neg_CE50"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "negative") & (x["CE"] == "50.0"),
    ## "cid_neg_CE70"  : lambda x: (x["fragmentation_method"] == "cid") & (x["ionMode"] == "negative") & (x["CE"] == "70.0"),
    ## "hcd_pos_CE30"  : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "positive") & (x["CE"] == "30.0"),
    ## "hcd_pos_CE50"  : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "positive") & (x["CE"] == "50.0"),
    ## "hcd_pos_CE70"  : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "positive") & (x["CE"] == "70.0"),
    "hcd_neg_step45.0" : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] in ["45.0", "stepped20,45,70ev(absolute)"]),
    "hcd_pos_step45.0" : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "positive") & (x["CE"] in ["45.0", "stepped20,45,70ev(absolute)"]),
    #"hcd_neg_CE20"     : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] == "20.0"),
    #"hcd_neg_CE30"     : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] == "30.0"),
    #"hcd_neg_CE40"     : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] == "40.0"),
    ## "hcd_neg_CE50"  : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] == "50.0"),
    ## "hcd_neg_CE70"  : lambda x: (x["fragmentation_method"] == "hcd") & (x["ionMode"] == "negative") & (x["CE"] == "70.0"),
}


# derived, do not change
colors = {ds: datasets[ds]["colour"] for ds in datasets}
# fmt: on

## Execute pipeline

In [3]:
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

pickle_file = f"{output_dir}/df_embeddings.pkl"
if os.path.exists(pickle_file):
    df = pd.read_pickle(pickle_file)

else:
    # Import the spectra and process MS2DeepScore embeddings
    df = generate_ms2deepscore_embeddings(model_file_name, datasets, data_to_add)
    
    # add associated metadata
    df = add_mzmine_metainfos(datasets, df)
    # df = add_sirius_fingerprints(datasets, df)
    df = add_sirius_canopus(datasets, df)
    df = add_sirius_predictions(datasets, df)
    df = add_mzmine_quant(datasets, df)
    
    # show overview and plot
    show_dataset_overview(df)
    generate_embedding_plots(df, output_dir, colors)

    # export dataframe for re-use
    df.to_pickle(f"{output_dir}/df_embeddings.pkl")

# train and predict new datasets
df_train, df_validation, df_inference, df_metrics = train_and_classify(df, subsets=training_subsets)
generate_prediction_overview(df, df_train, output_dir, "training", min_prediction_threshold=13)
generate_prediction_overview(df, df_inference, output_dir, "inference", min_prediction_threshold=13)

# Generate an overview of the machine learning metrics
generate_ml_metrics_overview(df_metrics, output_dir)



Training models
#######################################################
15 different models will be trained
Size of training dataset: 12602
Size of validation dataset: 0
Size of inference dataset: 1188



********************************************************************************
Subset: hcd_neg_step45.0
Number of spectra in subset (train): 317, these are 2.52% of the total spectra.
Overview of trainSubset_y_gt (ground-truth labels):
   - other: 264
   - relevant: 53

--------------------------------------------------------------------------------
Classifier: Nearest Neighbors n=3

   [Fold 1] Score: 0.938, Duration: 0.00 seconds
   - [Test subset] Confusion matrix relevant (6/32) -> relevant (6): 100.000%
   - [Test subset] Confusion matrix relevant (6/32) -> other (0): 0.000%
   - [Test subset] Confusion matrix other (26/32) -> relevant (2): 7.692%
   - [Test subset] Confusion matrix other (26/32) -> other (24): 92.308%
   - [Test subset] Balanced accuracy: 0.962
   - [Test su

i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: Li

   [Fold 1] Score: 0.688, Duration: 0.13 seconds
   - [Test subset] Confusion matrix relevant (6/32) -> relevant (0): 0.000%
   - [Test subset] Confusion matrix relevant (6/32) -> other (6): 100.000%
   - [Test subset] Confusion matrix other (26/32) -> relevant (4): 15.385%
   - [Test subset] Confusion matrix other (26/32) -> other (22): 84.615%
   - [Test subset] Balanced accuracy: 0.423
   - [Test subset] Precision: 0.638
   - [Test subset] F1 Score: 0.662
   - [Test subset] Recall: 0.688
   - [Test subset] ROC AUC: 0.423

   [Fold 2] Score: 0.719, Duration: 0.13 seconds
   - [Test subset] Confusion matrix relevant (6/32) -> relevant (1): 16.667%
   - [Test subset] Confusion matrix relevant (6/32) -> other (5): 83.333%
   - [Test subset] Confusion matrix other (26/32) -> relevant (4): 15.385%
   - [Test subset] Confusion matrix other (26/32) -> other (22): 84.615%
   - [Test subset] Balanced accuracy: 0.506
   - [Test subset] Precision: 0.700
   - [Test subset] F1 Score: 0.709
   - [

i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\sklearn\discriminant_analysis.py:1024: Li

   [Fold 3] Score: 0.781, Duration: 0.13 seconds
   - [Test subset] Confusion matrix relevant (6/32) -> relevant (2): 33.333%
   - [Test subset] Confusion matrix relevant (6/32) -> other (4): 66.667%
   - [Test subset] Confusion matrix other (26/32) -> relevant (3): 11.538%
   - [Test subset] Confusion matrix other (26/32) -> other (23): 88.462%
   - [Test subset] Balanced accuracy: 0.609
   - [Test subset] Precision: 0.767
   - [Test subset] F1 Score: 0.773
   - [Test subset] Recall: 0.781
   - [Test subset] ROC AUC: 0.609

   [Fold 4] Score: 0.844, Duration: 0.13 seconds
   - [Test subset] Confusion matrix relevant (5/32) -> relevant (1): 20.000%
   - [Test subset] Confusion matrix relevant (5/32) -> other (4): 80.000%
   - [Test subset] Confusion matrix other (27/32) -> relevant (1): 3.704%
   - [Test subset] Confusion matrix other (27/32) -> other (26): 96.296%
   - [Test subset] Balanced accuracy: 0.581
   - [Test subset] Precision: 0.809
   - [Test subset] F1 Score: 0.814
   - [T

...

Umap plot saved as ../../..//output_PrenylatedCompounds_BOKUDBs/training_relevant_predictions_umap.pdf


i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 16 x 12 in image.
i:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\.venv\Lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: ../../..//output_PrenylatedCompounds_BOKUDBs/training_relevant_predictions_feature_map.pdf


Feature map plot saved as ../../..//output_PrenylatedCompounds_BOKUDBs/training_relevant_predictions_feature_map.pdf


I:\Tomas_PrenylatedCompounds\analysis\classification\Package4Paper\AnnoMe\src\AnnoMe\Classification.py:1871: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


Saved table to ../../..//output_PrenylatedCompounds_BOKUDBs/training_data.xlsx


Generating Prediction Overview
#######################################################
plot saved as ../../..//output_PrenylatedCompounds_BOKUDBs/inference_relevant_predictions_classificationChart.pdf
Umap plot saved as ../../..//output_PrenylatedCompounds_BOKUDBs/inference_relevant_predictions_umap.pdf
Feature map plot saved as ../../..//output_PrenylatedCompounds_BOKUDBs/inference_relevant_predictions_feature_map.pdf
Saved table to ../../..//output_PrenylatedCompounds_BOKUDBs/inference_data.xlsx


Generating Machine Learning Metrics Overview
#######################################################
Machine learning metrics overview saved to ../../..//output_PrenylatedCompounds_BOKUDBs/ML_metrics_overview.tsv

Machine Learning Metrics and Sets Overview:
['Confusion matrix count: relevant -> relevant'
 'Confusion matrix percent: relevant -> relevant'
 'Confusion matrix count: relevant -> other'
 'Confusion m